In [2]:
import os
import sys
import requests
import sqlite3
import schedule
import time
# sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import *
# from inner_libs.libs import *
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
from libs.geral.myconstants import *
from libs.geral.utils import *
from libs.geral.df_pipes import *
from libs.denodo.conexaoDenodo import conexao_denodo as cnn

In [3]:
# Defina o caminho do diretório onde você deseja salvar os arquivos
diretorio = r'C:\rafa\bd_dolar'

In [4]:
# Função para criar o banco de dados e a tabela
def create_database():
    caminho_db = os.path.join(diretorio, 'cotacao_dolar.db')
    conn = sqlite3.connect(caminho_db)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS cotacoes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            data_hora TEXT,
            valor REAL
        )
    ''')
    conn.commit()
    conn.close()

In [5]:
# Função para obter a cotação do dólar
def get_cotacao_dolar():
    url = 'https://api.exchangerate-api.com/v4/latest/USD'  # Exemplo de API
    response = requests.get(url)
    data = response.json()
    valor_dolar = data['rates']['BRL']  # Cotação do dólar em relação ao real
    return valor_dolar

In [6]:
# Função para salvar a cotação no banco de dados
def save_cotacao():
    valor_dolar = get_cotacao_dolar()
    data_hora = time.strftime('%Y-%m-%d %H:%M:%S')
    
    # Salvar no banco de dados
    caminho_db = os.path.join(diretorio, 'cotacao_dolar.db')
    conn = sqlite3.connect(caminho_db)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO cotacoes (data_hora, valor) VALUES (?, ?)
    ''', (data_hora, valor_dolar))
    conn.commit()
    conn.close()
    print(f'Cotação salva automaticamente: {data_hora} - R$ {valor_dolar:.4f}')

In [7]:
# Função para salvar manualmente e exportar para Parquet
def save_manual_and_export():
    valor_dolar = get_cotacao_dolar()
    data_hora = time.strftime('%Y-%m-%d %H:%M:%S')
    
    # Salvar no banco de dados
    caminho_db = os.path.join(diretorio, 'cotacao_dolar.db')
    conn = sqlite3.connect(caminho_db)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO cotacoes (data_hora, valor) VALUES (?, ?)
    ''', (data_hora, valor_dolar))
    conn.commit()
    conn.close()
    # Criar DataFrame e exportar para Parquet com nome único
    df = pd.DataFrame({'data': [data_hora], 'valor cotação': [f'{valor_dolar:.4f}' ]})
    
    # Gerar um nome de arquivo único baseado na data e hora
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    caminho_parquet = os.path.join(diretorio, f'cotacao_dolar_{timestamp}.parquet')
    
    df.to_parquet(caminho_parquet, index=False, engine='pyarrow', compression='snappy')
    print(f'Cotação salva manualmente e exportada para Parquet: {data_hora} - R$ {valor_dolar:.4f}')


In [10]:
# Criar o banco de dados e a tabela
create_database()

# Agendar a execução da função a cada minuto
schedule.every(15).minutes.do(save_manual_and_export)

# Loop para manter o agendamento
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)

In [11]:
# Para rodar o agendamento em um thread separado
import threading
threading.Thread(target=run_schedule, daemon=True).start()

# save_manual_and_export()